# Fetch user metadata, posts with the [TikAPI](https://tikapi.io/documentation/)

#### Load Python tools and Jupyter config

In [1]:
import os
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
from tqdm.notebook import tqdm
from tikapi import TikAPI, ValidationException, ResponseException

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
mykey = os.environ.get("tikapikey")
api = TikAPI(mykey)

In [4]:
today = pd.Timestamp("today").strftime("%Y%m%d")

---

## Read data

#### Users names for API

In [5]:
users = [
    "famousmisspeaches",
    "stoolpresidente",
    "jeffjacksonnc",
]

#### Function to fetch metadata from a list of users

In [6]:
def fetch_user_meta(api, users):
    user_meta = []
    for u in tqdm(users):
        try:
            response = api.public.check(username=u).json()
            user_info = response["userInfo"]["user"]
            stats = response["userInfo"]["stats"]

            user_dict = {
                "id": user_info["id"],
                "name": user_info["nickname"],
                "name": user_info["nickname"],
                "followers": stats["followerCount"],
                "following": stats["followingCount"],
                "friend_count": stats["friendCount"],
                "heart_count": stats["heartCount"],
                "videos_posted": stats["videoCount"],
                "bio": user_info["signature"],
                "link": user_info.get("bioLink", {}).get("link", ""),
                "desc": response["shareMeta"]["desc"],
                "sec_id": user_info["secUid"],
            }

            user_meta.append(user_dict)
        except KeyError as e:
            print(f"Error processing user {u}: Missing key {e}")
        except Exception as e:
            print(f"Unexpected error processing user {u}: {e}")

    return user_meta

#### Fetch data

In [7]:
user_data = fetch_user_meta(api, users)

  0%|          | 0/3 [00:00<?, ?it/s]

#### Into a dataframe

In [8]:
df = pd.DataFrame(user_data)

In [9]:
df.head()

,id,name,followers,following,friend_count,heart_count,videos_posted,bio,link,desc,sec_id
0,7335986578207949870,Miss Peaches,498700,1,1,7100000,98,I'm Miss Peaches. My papa rescued me. I went from the outhouse to the penthouse,https://store.barstoolsports.com/products/adopt-don-t-shop-tee-1?variant=41508075044961,"@famousmisspeaches 498.7k Followers, 1 Following, 7.1m Likes - Watch awesome short videos created by Miss Peaches",MS4wLjABAAAAWm6gfQEeq-VWzulznJOtpkRrbbmXG2nAj3pnZWh5jwKA-B4xf2Y4ekOgx9lUbRy7
1,6659752019493208069,Dave Portnoy,3700000,62,43,170600000,1838,El Presidente/Barstool Sports Founder.,https://store.barstoolsports.com/products/ny-finest-tee?variant=41556376846433,"@stoolpresidente 3.7m Followers, 62 Following, 170.6m Likes - Watch awesome short videos created by Dave Portnoy",MS4wLjABAAAAINC_ElRR-l1RCcnEjOZhNO-9wOzAMf-YHXqRY8vvG9bEhMRa6iu23TaE3JPZYXBD
2,6909560330024731653,Jeff Jackson,2200000,936,440,36500000,104,"Dad, Soldier, Congressman (NC-14)",JeffJacksonNC.com/,"@jeffjacksonnc 2.2m Followers, 936 Following, 36.5m Likes - Watch awesome short videos created by Jeff Jackson",MS4wLjABAAAAZOQYqlJDh_Q3Nm1-wVs_8aseMBdfykKetnV5qH5ux4NBqjaDMi8fuKq_JXiPzi33


---

## Posts

#### Function to fetch a user's posts

In [10]:
def fetch_user_posts(sec_uid):
    all_posts = []

    try:
        response = api.public.posts(secUid=sec_uid)
        while response:
            posts_data = response.json()
            for item in posts_data.get("itemList", []):
                stats = item.get("stats", {})
                post_info = {
                    "user_id": item["author"]["id"],
                    "user_name": item["author"]["nickname"],
                    "user_unique_id": item["author"]["uniqueId"],
                    "post_desc": item["desc"],
                    "create_time": item["createTime"],
                    "post_id": item["video"]["id"],
                    "duration": item["video"].get("duration", "N/A"),
                    "bookmark_count": stats.get("collectCount", 0),
                    "comment_count": stats.get("commentCount", 0),
                    "heart_count": stats.get("diggCount", 0),
                    "share_count": stats.get("shareCount", 0),
                    "play_count": stats.get("playCount", 0),
                }
                all_posts.append(post_info)

            if not posts_data.get("hasMore", False):
                break

            response = response.next_items()

    except Exception as e:
        print(f"An error occurred: {e}")

    return pd.DataFrame(all_posts)

#### Define IDs for users

In [11]:
ids = df["sec_id"].to_list()

#### Get a list of dataframes for each user's posts

In [12]:
dfs = []

for i in ids:
    dfs.append(fetch_user_posts(i))

An error occurred: Something went wrong.


#### Clean up the dates. All datetimes Eastern.

In [13]:
df_posts = pd.concat(dfs)

In [14]:
df_posts["create_time_est"] = (
    pd.to_datetime(df_posts["create_time"], unit="s")
    .dt.tz_localize("UTC")
    .dt.tz_convert("America/New_York")
)

In [15]:
df_posts["created_date"] = pd.to_datetime(df_posts["create_time_est"], unit="s").dt.date
df_posts["created_time"] = pd.to_datetime(df_posts["create_time_est"], unit="s").dt.time
df_posts["create_time"] = pd.to_datetime(df_posts["create_time_est"], unit="s")

In [16]:
df_posts = df_posts.drop("create_time", axis=1)

---

## Exports

#### Out put json and csv for each user

In [17]:
for user_to_export in df_posts["user_unique_id"].unique():
    df_posts.query(f'user_unique_id == "{user_to_export}"').to_json(
        f"data/processed/{user_to_export}_all_posts.json",
        indent=4,
        orient="records",
    )

    df_posts.query(f'user_unique_id == "{user_to_export}"').to_csv(
        f"data/processed/{user_to_export}_all_posts.csv", index=False
    )